<a href="https://colab.research.google.com/github/Hyun-Jun-Lee/Bank-Churner-Prediction/blob/master/Bank_Churner_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 데이터 불러오기 및 EDA


In [ ]:
from google.colab import drive 

drive.mount('/content/gdrive/')

In [ ]:
ls

라이브러리 모음

In [ ]:

import sys

if 'google.colab' in sys.modules:
    # Install packages in Colab
    !pip install -U pandas-profiling

In [ ]:
import pandas as pd
import numpy as np
import pandas_profiling
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import plotly.offline as pyo

In [ ]:
data = pd.read_csv('/content/gdrive/MyDrive/BankChurners.csv')
data.head()

In [ ]:
pr = data.profile_report()
pr

### 변수 설명

- CLIENTNUM : 고객 번호. 계정을 보유한 고객의 고유 식별자
- **Attrition_Flag(Target) : 고객 이탈 여부, 1 - Churner/ 0 - non Churner**
- Customer_Age : 고객의 나이
- Gender : 성별, M=Male, F=Female
- Dependent_count : 부양가족수
- Education_Level : 고객의 학력
- Marital_Status : Married, Single, Divorced, Unknown
- Income_Category : 연간 소득 (< 40K, 40K - 60K, 60K - 80K, 80K-120K, >)
- Card_Category : 카드 등급 (Blue, Silver, Gold, Platinum)
- Months_on_book : 은행과 거래 기간
- Total_Relationship_Count : 고객이 보유한 총 상품 수
- Months_Inactive_12_mon : 최근 12개월 동안 비활성 개월 수
- Contacts_Count_12_mon : 최근 12개월 동안 연락한 횟수
- Credit_Limit : 신용 한도
- Total_Revolving_Bal : 총 리볼빙(일부결제금액이월약정) 잔액
- Avg_Open_To_Buy : Open to Buy Credit Line (Average of last 12 months)
- Total_Amt_Chng_Q4_Q1 : 거래 금액의 변화 (1분기 대비 4분기)
- Total_Trans_Amt : 최근 12개월 총 거래 금액
- Total_Trans_Ct : 최근 12개월 총 거래 횟 수
- Total_Ct_Chng_Q4_Q1 : Change in Transaction Count (Q4 over Q1)
- Avg_Utilization_Ratio : 평균 카드 사용률

### EDA

In [ ]:
# Customer_Age 분포

# 그래프 영역 나누기
f = make_subplots(rows=2, cols=1) 

# boxplot과 histogram 그래프 x,y축 지정, boxmean=평균값표시
t1 = go.Box(x=data['Customer_Age'], name='Box Plot for Customer_age', boxmean=True)
t2 = go.Histogram(x=data['Customer_Age'], name='Histogram Customer_age')

# plotly에서는 add_trace() 함수로 지정
f.add_trace(t1, row=1, col=1) 
f.add_trace(t2, row=2, col=1)

f.update_layout(height=600, width=1000, title_text = 'Customer_Age')

Customer_Age는 정규분포 <br>
아직 경제활동을 하고있지 않는 10대 이하와 70세 이상의 초고령층의 데이터가 필요할까?

In [ ]:
data.sort_values(by='Customer_Age', ascending=False)

In [ ]:
# Card_Category 분포
%matplotlib inline 

plt.figure(figsize=(8,8))
sns.countplot(data=data, x='Card_Category', hue='Gender')
plt.title('Card Rating by Gender')
plt.show()

In [ ]:
data['Card_Category'].value_counts()

In [ ]:
# Dependent_count 분포

# 그래프 영역 나누기
f = make_subplots(rows=2, cols=1) 

# boxplot과 histogram 그래프 x,y축 지정, boxmean=평균값표시
t1 = go.Box(x=data['Dependent_count'], name='Box Plot for Dependent_count', boxmean=True)
t2 = go.Histogram(x=data['Dependent_count'], name='Histogram Dependent_count')

# plotly에서는 add_trace() 함수로 지정
f.add_trace(t1, row=1, col=1) 
f.add_trace(t2, row=2, col=1)

f.update_layout(height=600, width=1000, title_text = 'Dependent_count')

In [ ]:
# Education_Level 분포
ex.pie(data,names='Education_Level',title='Propotion Of Education Levels',hole=0.2)

In [ ]:
data['Education_Level'].value_counts()

In [ ]:
# Marital_Stattus 분포

px.pie(data,names='Marital_Status',title='Propotion Of Different Marriage Statuses',hole=0.33)

In [ ]:
# Income_Category 분포

px.pie(data,names='Income_Category',title='Income_Category',hole=0.33)

In [ ]:
# 'Atrittion_Flag' 타겟 변수 분포 
px.pie(data,names='Attrition_Flag',title='churn vs not churn customers',hole=0.33)

# Data Preprocessing

In [ ]:
data.head()

In [ ]:
# 결측 값이 unknown 으로 되어 있어서 na값 확인되지 않음
data.isna().sum()

In [ ]:
# 필요없는 변수 제거
data.drop(columns=['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1',
                   'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2',
                   'CLIENTNUM'], inplace=True)

In [ ]:
data

In [ ]:
# Unknown 비율 20% 이하 제거

data = data[data['Marital_Status'] != 'Unknown'] # 7.4%
data = data[data['Income_Category'] != 'Unknown'] # 11%
data = data[data['Education_Level'] != 'Unknown'] # 15%

print(data['Marital_Status'].value_counts())
print('\n')
print(data['Income_Category'].value_counts())
print('\n')
print(data['Education_Level'].value_counts())

### Encoding

In [ ]:
# one-hot encoding
data['Attrition_Flag'] = data['Attrition_Flag'].replace({'Attrited Customer':1, 'Existing Customer':0})
data['Gender'] = data['Gender'].replace({'F':1, 'M':0})
data = pd.get_dummies(data)

In [ ]:
# Oridinal Encoding

Income_Category_map = {
    'Less than $40K' : 0,
    '$40K - $60K'    : 1,
    '$60K - $80K'    : 2,
    '$80K - $120K'   : 3,
    '$120K +'        : 4
}


Card_Category_map = {
    'Blue'     : 0,
    'Silver'   : 1,
    'Gold'     : 2,
    'Platinum' : 3
}

Education_Level_map = {
    'Uneducated'    : 0,
    'High School'   : 1,
    'College'       : 2,
    'Graduate'      : 3,
    'Post-Graduate' : 4,
    'Doctorate'     : 5
    }

data.loc[:, 'Income_Category'] = data['Income_Category'].map(Income_Category_map)
data.loc[:, 'Card_Category'] = data['Card_Category'].map(Card_Category_map)
data.loc[:, 'Education_Level'] = data['Education_Level'].map(Education_Level_map)


In [ ]:
data.head()